In [8]:
import os 
import modal
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
from data import items
import chromadb
import pickle
import sys
from sentence_transformers import SentenceTransformer
from openai import OpenAI
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [9]:
openai = OpenAI()

In [10]:
sys.path.append(os.path.abspath("data"))
with open('data/train.pkl', 'rb') as file:
    train = pickle.load(file)

with open('data/test.pkl', 'rb') as file:
    test = pickle.load(file)


In [11]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
DB = "products_vectorstore"

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
!modal deploy price_service.py

╭─ Modal Deprecation Warning (2025-01-15) ─────────────────────────────────────╮
│ The `@modal.build` decorator is deprecated and will be removed in a future   │
│ release.                                                                     │
│                                                                              │
│ We now recommend storing large assets (such as model weights) using a        │
│ `modal.Volume` instead of writing them directly into the `modal.Image`       │
│ filesystem. For other use cases we recommend using `Image.run_function`      │
│ instead.                                                                     │
│                                                                              │
│ See https://modal.com/docs/guide/modal-1-0-migration for more information.   │
│                                                                              │
│ Source: /Users/amir/Desktop/GitHub/AgenticAI/price_service.py:34             │
│   @modal.build()          

In [12]:
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('products')


In [14]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [24]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n","")
    return text.split("\n\nPrice is $")[0]

In [12]:
for i in tqdm(range(0,len(train),1000)):
    documents = [description(item) for item in train[i:i+1000]]
    vectors = model.encode(documents)
    metadata = [{'category': item.category, 'price':item.price} for item in train[i:i+1000]]
    ids = [ f"doc_{j}" for j in range(i,i+1000)]
    collection.add(
        ids = ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadata
    )

100%|██████████| 400/400 [14:23<00:00,  2.16s/it]


In [13]:
collection.count()

400000

In [10]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [11]:
def vector(item):
    return model.encode([description(item)])

In [12]:
def find_similars(item):
    results = collection.query(query_embeddings=vector(item).astype(float).tolist(),n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents,prices

In [15]:
documents, prices = find_similars(test[10])

In [16]:
print(make_context(documents, prices))

To provide some context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Viking Horns On-Board AIR System with 1/2 Air Hose & Electric Solenoid, 5 Gallon Air Tank and 200 PSI Air Compressor Kit, for Train Air Horns
Viking Horns 5 gallon Air Tank & 200 PSI H.D Air Compressor Kit. For high pressure air horns systems that require an on-board air system. Air tank made from heavy gauge steel for outdoor use. Made to hold up to 250 psi air pressure. Comes with psi air gauge, psi air pressure switch, compression fittings for 1/2 O.D air hose, air pressure release valve, 1/2 heavy duty electric air valve solenoid, 1/2 high pressure air hose, 200 PSI Heavy Duty 100% duty cycle air
Price is $298.65

Potentially related product:
Viking Horns Loud Sound 149 Decibels Train Air Horn Kit with 1 Gallon Air Tank
Viking Horns brand, Air Tank & 12 volt Compressor air supply system. This Air kit is designed for use with any high-pressure Air 

In [17]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [18]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollat", "").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]



In [19]:
print(train[10].test_prompt())

How much does this cost to the nearest dollar?

PG Engine Air Filter | Fits Chevrolet Camaro
SUPERIOR ENGINE PROTECTION – Premium Guard Air Filters filter out 99% of incoming engine air to help extend the life of your engine. ENHANCED PERFORMANCE – High-capacity air filter media removes dangerous particles improving engine performance and increasing engine efficiency. EASY TO INSTALL - Premium Guard Air Filters are engineered to fit perfectly inside your vehicle’s housing for quick and easy installation. Compatible with Chevrolet Camaro. Precisely designed, engineered, and tested to meet and exceed all GENERAL MOTORS OE air filter requirements. Replaces GENERAL MOTORS Air Filter. Always check fitment using the Vehicle Filter Manufacturer Premium Guard, Brand Premium Guard, Weight 1.12 pounds, Dimensions 12.1 x 10.6 x 2.1 inches

Price is $


In [20]:
print(messages_for(test[0], documents, prices)[1]['content'])

To provide some context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Viking Horns On-Board AIR System with 1/2 Air Hose & Electric Solenoid, 5 Gallon Air Tank and 200 PSI Air Compressor Kit, for Train Air Horns
Viking Horns 5 gallon Air Tank & 200 PSI H.D Air Compressor Kit. For high pressure air horns systems that require an on-board air system. Air tank made from heavy gauge steel for outdoor use. Made to hold up to 250 psi air pressure. Comes with psi air gauge, psi air pressure switch, compression fittings for 1/2 O.D air hose, air pressure release valve, 1/2 heavy duty electric air valve solenoid, 1/2 high pressure air hose, 200 PSI Heavy Duty 100% duty cycle air
Price is $298.65

Potentially related product:
Viking Horns Loud Sound 149 Decibels Train Air Horn Kit with 1 Gallon Air Tank
Viking Horns brand, Air Tank & 12 volt Compressor air supply system. This Air kit is designed for use with any high-pressure Air 

In [21]:
def gpt4o_mini(item):
    documents,prices = find_similars(item)
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages_for(item, documents, prices),
        seed=42,
        max_tokens=5
    )
    reply = response.choices[0].message.content
    return get_price(reply) 

In [24]:
gpt4o_mini(test[10])

328.95

In [25]:
test[0].prompt

"How much does this cost to the nearest dollar?\n\nOEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K\n\nPrice is $374.00"

In [17]:
from agents.frontier_agent import FrontierAgent
from agents.ensemble_agent import RandomForestAgent
from agents.price_agent import SpecialistAgent
import logging

In [18]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [19]:
frontieragent = FrontierAgent(collection)
specialistagent = SpecialistAgent()
random_forest = RandomForestAgent()

INFO:root:[Frontier Agent]
INFO:root:[Frontier Agent]
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent]
INFO:root:[SpecialistAgent]
/opt/anaconda3/envs/llms/lib/python3.11/asyncio/events.py:84: DeprecationError: 2025-01-27: `modal.Cls.lookup` is deprecated and will be removed in a future release. It can be replaced with `modal.Cls.from_name`.

See https://modal.com/docs/guide/modal-1-0-migration for more information.
  self._context.run(self._callback, *self._args)
INFO:root:[SpecialistAgent]
INFO:root:[Random Forest Agent]
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Random Forest Agent]


In [21]:
# result = collection.get(include=['embeddings', 'documents', 'metadatas'])
# vectors = np.array(result['embeddings'])
# documents = result['documents']
# prices = [metadata['price'] for metadata in result['metadatas']]

# rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
# rf.fit(vectors,prices) 

In [33]:
joblib.dump(rf, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [22]:
product = "Quadcast HyperX condenser mic for high quality audio for podcasting"


print(specialistagent.price(product))
print(frontieragent.price(product))
print(random_forest.price(product))

INFO:root:[SpecialistAgent]
INFO:root:[SpecialistAgent]
INFO:root:[Frontier Agent]


189.0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent]
INFO:root:[Frontier Agent]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent]
INFO:root:[Random Forest Agent]


195.19


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Random Forest Agent]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Random Forest Agent]


280.6440000000002


In [25]:
specialists = []
frontiers = []
random_forests = []
prices = []
for item in tqdm(test[1000:1250]):
    text = description(item)
    specialists.append(specialistagent.price(text))
    frontiers.append(frontieragent.price(text))
    random_forests.append(random_forest.price(text))
    prices.append(item.price)
mins = [min(s,f,r) for s,f,r in zip(specialists, frontiers, random_forests)]
maxes = [max(s,f,r) for s,f,r in zip(specialists, frontiers, random_forests)]

X = pd.DataFrame({
    'Specialist': specialists,
    'Frontier': frontiers,
    'RandomForest': random_forests,
    'Min': mins,
    'Max': maxes,
})

# Convert y to a Series
y = pd.Series(prices)
# Train a Linear Regression
np.random.seed(42)

lr = LinearRegression()
lr.fit(X, y)

feature_columns = X.columns.tolist()

for feature, coef in zip(feature_columns, lr.coef_):
    print(f"{feature}: {coef:.2f}")
print(f"Intercept={lr.intercept_:.2f}")
joblib.dump(lr, 'ensemble_model.pkl')

INFO:root:[SpecialistAgent]
INFO:root:[Frontier Agent]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent]
INFO:root:[Frontier Agent]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent]
INFO:root:[Random Forest Agent]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Random Forest Agent]


...

INFO:root:[Random Forest Agent]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Random Forest Agent]
100%|██████████| 250/250 [10:43<00:00,  2.58s/it]

Specialist: 0.62
Frontier: 0.39
RandomForest: -0.11
Min: 0.04
Max: 0.03
Intercept=26.65


['ensemble_model.pkl']

In [26]:
from agents.ensemble_agent import EnsembleAgent
ensemble = EnsembleAgent(collection)
product = "Quadcast HyperX condenser mic for high quality audio for podcasting"

ensemble.price(product)


INFO:root:[Ensemble Agent]
INFO:root:[SpecialistAgent]
/opt/anaconda3/envs/llms/lib/python3.11/asyncio/events.py:84: DeprecationError: 2025-01-27: `modal.Cls.lookup` is deprecated and will be removed in a future release. It can be replaced with `modal.Cls.from_name`.

See https://modal.com/docs/guide/modal-1-0-migration for more information.
  self._context.run(self._callback, *self._args)
INFO:root:[SpecialistAgent]
INFO:root:[Frontier Agent]
INFO:root:[Frontier Agent]
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent]
INFO:root:[Random Forest Agent]
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Random Forest Agent]
INFO:root:[Ensemble Agent]
INFO:root

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent]
INFO:root:[Frontier Agent]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent]
INFO:root:[Random Forest Agent]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Random Forest Agent]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Random Forest Agent]
INFO:root:[Ensemble Agent]


205.0205173065032